In [ ]:
from tinc import *
tclient = TincClient()

In [ ]:
tclient.print()

In [ ]:
tclient.parameter_spaces[0].get_current_relative_path()

In [ ]:
tclient.parameter_spaces[0].get_root_path()

The 'graph' ```imageBuffer``` determines what graph is shown.

In [ ]:
imageBuffer = tclient.get_disk_buffer('graph')

In [ ]:
imageBuffer._path

In [ ]:
import random
import matplotlib.pyplot as plt
import threading

def update_graph(parameter_value):
    print("Parameter value " + str(parameter_value))
    data = [random.random() * parameter_value for i in range(10)]
    with threading.Lock():
        fname = imageBuffer.get_filename_for_writing()
        f = plt.figure()
        plt.title(f" Time: {parameter_value}")
        plt.plot(data)
        #print("Update " + fname)
        plt.savefig(fname)
        # The following to avoid extra printout in jupyter cell
        plt.close()
        f.clf()
        del f
        imageBuffer.done_writing_file(fname)

In [ ]:
update_graph(40)

In [ ]:
shellsites = tclient.get_parameter("ShellSiteTypes")

In [ ]:
shellsites.elements

In [ ]:
shellsites.get_current_elements()

In [ ]:
shellsites.register_callback(update_graph)

In [ ]:
shellsites.value

In [ ]:
ms = tclient.get_parameter("markerScale")

In [ ]:
ms.value

In [ ]:
ms.value = 20

In [ ]:
mc = tclient.get_parameter("markerColor")

In [ ]:
mc.value

In [ ]:
mc.value = [0.5, 1, 0.5, 1]

In [ ]:
import matplotlib

In [ ]:
matplotlib.colors.to_rgb('blue')

In [ ]:
colors = ['blue', 'red', 'green', 'white', 'darkBlue', 'cyan', 'magenta', 'lightBlue','blue', 'red', 'green', 'cyan', 'magenta', 'lightBlue']

In [ ]:
[matplotlib.colors.to_rgb(c) for c in colors]

In [ ]:
bin(shellsites.value)

In [ ]:
def set_color(shellSite):
    b = int(shellSite)
    for c in colors:
        if b & 1 == 1:
            rgb = list(matplotlib.colors.to_rgb(c))
            rgb.append(1)
            mc.value = rgb
            break
        b = b >> 1
    

In [ ]:
set_color(1024)

In [ ]:
shellsites.register_callback(set_color)

## Datapools

In [ ]:
[dp.id for dp in tclient.datapools]

In [ ]:
dp = tclient.get_datapool("resultsData")

In [ ]:
dp.slice_cache_dir

In [ ]:
dp.get_slice("<comp(a)>", "chempotA")

In [ ]:
dir_param = tclient.get_parameter("dir")

In [ ]:
dir_param.value

In [ ]:
dir_param.value = 70

In [ ]:
dir_param.value = 30
dp.get_slice("<comp(c)>", "chempotA")

In [ ]:
neighborhood = tclient.get_datapool("neighborhood")

In [ ]:
neighborhood

In [ ]:
neighborhood.get_current_files()

In [ ]:
dp.slice_cache_dir

In [ ]:
param=tclient.parameter_spaces[0]
root_path=param.get_root_path()
trajectories_path=root_path+param.get_current_path()+"trajectory.nc"

In [ ]:
trajectories_path

In [ ]:
import netCDF4
nc = netCDF4.Dataset(trajectories_path)
nc.variables['occupation_dofs']
#choose an atom to track
#find the first step that site index changes, record new position
#track that position from then on
# repeat if changes.
atom_to_track=0
steps_changed=[]
previous_dof=nc.variables['occupation_dofs'][2000].data
print(previous_dof)

In [ ]:
len(nc.variables['occupation_dofs'][2000].data)

In [ ]:
%pylab inline

In [ ]:
!ncdump -c C:\\Users\\Andres\\source\\repos\\vdv_data\\MgTiS_spinel_diffusion_visualization\\event_tracker_testing_quick\\24\\conditions.0\\trajectory.nc

In [ ]:
time = zeros((3000, 24696))

In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset(trajectories_path, chunks = {'sample_dim':10})

In [ ]:
equals = []
for i in range(len(ds['occupation_dofs']) -1):
    equals.append(ds['occupation_dofs'][i] == ds['occupation_dofs'][i + 1])

In [ ]:
%%timeit
argwhere(equals)

In [ ]:
list_of_occ_dofs=[]
for time_slice in nc.variables['occupation_dofs']:
    print (".")
    list_of_occ_dofs.append(time_slice)

In [ ]:
for step_count in range(nc.variables['occupation_dofs'].shape[0]):
    new_dof=nc.variables['occupation_dofs'][step_count].data
    if previous_dof[atom_to_track]!=new_dof[atom_to_track]:
        steps_changed.append(step_count)
        for i in len(previous_dof):
            if i!=atom_to_track and previous_dof[i]!=new_dof[i]:
                atom_to_track=i
               
    previous_dof=new_dof
   
print(steps_changed)

In [ ]:
tclient.synchronize()

In [ ]:
tclient.disk_buffers[-1].id

In [ ]:
traj_buffer = tclient.get_disk_buffer("trajectory_buffer")

In [ ]:
def write_positions(pos):
    import json
    fname = traj_buffer.get_filename_for_writing()

    with open(fname, 'w') as f:
        json.dump(pos, f)
        traj_buffer.done_writing_file(fname)

In [ ]:
pos = [[ [0.0,0.0, 0]], [ [0,0.003, 0]],[ [0.001,0.003, 0]], [ [-0.003,0.005, 0]], [ [-0.001,0.001, 0.004]]]
write_positions(pos)

In [ ]:
pos = [[ [0.0,0.0, 0]], [ [0,0.3, 0]],[ [0.1,0.3, 0]], [ [-0.3,0.5, 0]], [ [-0.1,0.1, 0.4]]]
write_positions(pos)

In [ ]:
pos = [[ [10.0,10.0, 10]], [ [0,5.3, 0]], [ [5.1,5.3, 0]], [ [-5.3,5.5, 0]], [ [-5.1,5.1, 5.4]]]
write_positions(pos)

In [ ]:
tclient.get_parameter("width", "trajectory").value = 2
tclient.get_parameter("alpha", "trajectory").value = 0.5